In [18]:
# # GPU allocation
# from gpuutils import GpuUtils
# GpuUtils.allocate(gpu_count=1, framework='keras')

# import tensorflow as tf
# physical_devices = tf.config.list_physical_devices('GPU')
# for device in physical_devices:
#     tf.config.experimental.set_memory_growth(device, True)
# # --------------

# Imports
import os
import numpy as np
import pickle
import argparse
from termcolor import colored
import time
from toolbox import load_file, find_68_interval, models_dir
from radiotools import helper as hp
from PIL import Image

#from scipy import stats
import wandb
from wandb.keras import WandbCallback
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, Flatten, Dropout
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, AveragePooling1D, Input, Flatten
from tensorflow.keras.layers import BatchNormalization, Lambda, MaxPooling2D, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence, plot_model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau

from generator import TrainDataset, ValDataset, n_events_per_file, n_files_train, batch_size
from constants import run_version, dataset_name, datapath, data_filename, label_filename, plots_dir, project_name, n_files, n_files_val, dataset_em, dataset_noise, test_file_ids
# -------

# Values
feedback_freq = 1 # Only train on 1/feedback_freq of data per epoch
architectures_dir = "architectures"
learning_rate = 0.00005
epochs = 100
loss_function = "mean_squared_error"
es_patience = 8
es_min_delta = 0.0001 # Old value: es_min_delta = 0.0001
# ------

# Parse arguments
# parser = argparse.ArgumentParser(description='Neural network for neutrino energy reconstruction')
# parser.add_argument("run_id", type=str ,help="the id of the run, eg '3.2' for run3.2")

# args = parser.parse_args()
run_id = "E9.TEST"

# Save the run name
run_name = f"run{run_id}"

# Make sure run_name is compatible with run_version
this_run_version = run_name.split(".")[0]
this_run_id = run_name.split(".")[1]
assert this_run_version == run_version, f"run_version ({run_version}) does not match the run version for this run ({this_run_version})"

# Models folder
saved_model_dir = models_dir(run_name)

# Make sure saved_models folder exists
if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)

# Make sure architectures folder exists
if not os.path.exists(f"{saved_model_dir}/{architectures_dir}"):
    os.makedirs(f"{saved_model_dir}/{architectures_dir}")


# Model params
conv2D_filter_size = 5
pooling_size = 2
amount_Conv2D_layers_per_block = 3 
amount_Conv2D_blocks = 4
conv2D_filter_amount = 32
activation_function = "relu"



# ----------- Create model -----------
model = Sequential()

# Conv2D block 1
model.add(Conv2D(conv2D_filter_amount, (1, conv2D_filter_size), strides=(1, 1), padding='same', activation=activation_function, input_shape=(5, 512, 1)))

for _ in range(amount_Conv2D_layers_per_block-1):
    model.add(Conv2D(conv2D_filter_amount, (1, conv2D_filter_size), strides=(1, 1), padding='same', activation=activation_function))

# MaxPooling to reduce size
model.add(AveragePooling2D(pool_size=(1, pooling_size)))

for i in range(amount_Conv2D_blocks-1):
    # Conv2D block
    for _ in range(amount_Conv2D_layers_per_block):
        model.add(Conv2D(conv2D_filter_amount*2**(i+1), (1, conv2D_filter_size), strides=(1, 1), padding='same', activation=activation_function))

    # MaxPooling to reduce size
    model.add(AveragePooling2D(pool_size=(1, pooling_size)))

model.add(Conv2D(256, (1, 2), strides=(1, 1), activation=activation_function))
# model.add(Conv2D(256, (5, 1), strides=(1, 1), activation=activation_function))

model.add(Reshape((5, 256)))
# model.add(Reshape((5, 256, 1)))

# model.add(Conv2D(256, (5, 1), strides=(1, 1), activation=activation_function))

# TimeDistributed(Conv1D)() skips the sharing of filters (kerners), or we do Conv2D

# y axis filters, x axis time (eg 5, 128, nfilters), 5 is the channels in the image (RGB would be 3)

# then fully conected in features, and maybe 3 in time, but thne many parameters


# Global average pooling/global max pooling


# Batnch normalizaiton between fully onnected layers)



# Alternative 1
# GO down to (5, 1, 256) and then do some Conv1D (or Conv2D) to (5, 256))
# then filter 2x3 Conv2D (2x3) 64 channels then Global Pooling


# Alternative 2 (gravitational approach)
# (128, 128, 5)
# t, features, antennas (after reshgape)
# then 5x5 filter, maybe 64 of them
# becomes  (128, 128, 64)

# 1st option
# 


# Batch normalization
model.add(BatchNormalization())

# Flatten prior to dense layers
model.add(Flatten())

# Dense layers (fully connected)
model.add(Dense(1024, activation=activation_function))
model.add(Dense(1024, activation=activation_function))
model.add(Dense(512, activation=activation_function))
model.add(Dense(256, activation=activation_function))
model.add(Dense(128, activation=activation_function))

# model.add(Dense(512, activation=activation_function))
# # model.add(Dropout(.1))
# model.add(Dense(256, activation=activation_function))
# # model.add(Dropout(.1))
# model.add(Dense(128, activation=activation_function))
# # model.add(Dropout(.1))

# Output layer
model.add(Dense(1))

model.compile(loss=loss_function,
              optimizer=Adam(lr=learning_rate))
model.summary()
# ------------------------------------

# Save the model (for opening in eg Netron)
#model.save(f'{saved_model_dir}/{architectures_dir}/model_architecture_{run_name}.h5')
plot_model(model, to_file=f'{saved_model_dir}/{architectures_dir}/model_architecture_{run_name}.png', show_shapes=True)
model_json = model.to_json()
with open(f'{saved_model_dir}/{architectures_dir}/model_architecture_{run_name}.json', "w") as json_file:
    json_file.write(model_json)




Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_170 (Conv2D)          (None, 5, 512, 32)        192       
_________________________________________________________________
conv2d_171 (Conv2D)          (None, 5, 512, 32)        5152      
_________________________________________________________________
conv2d_172 (Conv2D)          (None, 5, 512, 32)        5152      
_________________________________________________________________
average_pooling2d_52 (Averag (None, 5, 256, 32)        0         
_________________________________________________________________
conv2d_173 (Conv2D)          (None, 5, 256, 64)        10304     
_________________________________________________________________
conv2d_174 (Conv2D)          (None, 5, 256, 64)        20544     
_________________________________________________________________
conv2d_175 (Conv2D)          (None, 5, 256, 64)      